In [1]:
import glob

In [2]:
file_dir = r'data\**\*.txt'
file_paths = glob.glob(file_dir)
print(len(file_paths))

30


In [3]:
def readfile(filename):
  from functools import reduce
  with open(filename, 'r', encoding='utf-8') as f:
      lines = f.readlines()
      text = reduce(lambda a, b : a + " " +str(b), lines)
  return text

textlist = []
for file in file_paths:
  textlist.append(readfile(file))

In [4]:
from preprocess import normalize_text, remove_punctuations, tokenize_text, stopword_remover

In [5]:
file_paths[0]

'data\\agyaya\\cassendra ka abhishap(agyaya).txt'

In [7]:
text = remove_punctuations(textlist[0])
text = normalize_text(text)
tokens = tokenize_text(text)
tokens = stopword_remover(tokens)

एक खजूर के वृक्षों की छोटीसी छाया उस कड़ाके की धूप में मानो सिकुड़ कर अपनेआपमें या पेड़ के पैरों तले छिपी जा रही है अपनी उत्तप्त साँस से छटपटाते हुए वातावरण से दोचार केना के फूलों की आभा एक तरलता एक चिकनेपन का भ्रम उत्पन्न कर रही है यद्यपि है सब ओर सूनापन प्यासापन रुखाई उन केना के फूलों के पास ही एक छींट के टुकड़े से अपने कंधे ढँके हुए मेरिया बैठी है उससे कुछ ही दूर भूमि पर एक अख़बार बिछाए उसकी छोटी बहन कार्मेन एक रूमाल काढ़ रही है वे दोनों अपनेअपने ध्यान में मस्त हैं किंतु उनके ध्यान एक ही विषय को दो विभिन्न दृष्टियों से देख रहे हैं यद्यपि वे स्वयं इस बात को नहीं जानतीं कि उनके विचार एकदूसरे के कितने पास मँडरा रहे हैंयद्यपि मेरिया उसे कभी स्वीकार नहीं करेगी क्योंकि वह इसे अपने हृदय का गुप्ततम रहस्य समझती है कार्मेन की आँखें उसके हाथ के रूमाल पर लगी हुई हैं वह उस पर लाल धागे से एक नाम काढ़ रही है जो मेहँदी के रंग से उस पर लिखा हुआ हैमिगेल नाम के चारों ओर एक बेल काढ़ी जा चुकी है और बेल के ऊपर एक लाल झंडा मेरिया अपने पास की किसी चीज़ को अपने चर्मचक्षुओं से भी नहीं देख रही है केना के फूलो

In [ ]:
from sklearn.model_selection

In [9]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained mBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

c:\Users\dhruv\miniconda3\envs\temp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dhruv\miniconda3\envs\temp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
